# 0. Preparations

## 0.1. Import required packages

In [3]:
from time import time

import pandas as pd
import numpy as np

# Import the normal distribution function from the stats package
from scipy.stats import norm

# Import the accuracy_score method from sklearn to compute the accuracies of predictions
from sklearn.metrics import accuracy_score

## 0.2. Define general functions

In [4]:
# Define which indices and headings to use for the timing DataFrames
timing_index = ['train', 'test', 'total']
timing_columns = ['fold 1', 'fold 2', 'fold 3', 'fold 4', 'fold 5', 'fold 6', 'fold 7', 'fold 8', 'fold 9', 'fold 10', 'mean', 'std']

In [5]:
def cross_validation_train_test_split(data_set, buckets_location, bucket_number):
    # Read all the bucket indices for the specified bucket number from the provided CSV file
    bucket_indices = pd.read_csv(buckets_location).iloc[bucket_number].dropna().values

    # Split the entire data set into testing and training data
    data_test = data_set.iloc[bucket_indices]
    data_train = data_set.drop(bucket_indices, axis='index')

    return data_train, data_test

def cross_validation(all_data, target, buckets_location, e, enhanced):
   
    # Define an empty list to store the accuracies in
    accuracies = []
    
    # Define empty lists and a DataFrame to store the timing measurements in
    training_times = []
    testing_times = []
    timings = pd.DataFrame(index=timing_index, columns=timing_columns)
    
    # Run the classification with 10-fold cross-validation
    for i in range(0, 10):

        # Retrieve the training and testing data according to a specific bucket number (starts from 0)
        training_data, testing_data = cross_validation_train_test_split(all_data, buckets_location=buckets_location, bucket_number=i)
        
        # Split the data into features and labels
        features_train = training_data.drop(target, axis='columns')
        features_test = testing_data.drop(target, axis='columns')
        
        labels_train = pd.DataFrame(training_data[target])
        labels_test = pd.DataFrame(testing_data[target])
        
        # Instantiate a new instance of the classifier
        classifier = CustomNaiveBayes(e, enhanced)
        
        
        
        # --- Start training time --- #
        t_train_start = time()

        # Train the classifier with the given data
        classifier.fit(features_train, labels_train)

        t_train_end = time()
        # --- End training time --- #  
        
        
        
        # --- Start prediction time ---#
        t_predict_start = time()

        # Make predictions with the classifier
        predictions = classifier.predict(features_test)

        t_predict_end = time()
        # --- End prediction time --- #

        
    
        # Compute the accuracy for the classifier
        accuracy = accuracy_score(labels_test, predictions)

        # Add the accuracy to the list of accuracies
        accuracies.append(accuracy)
        
        # Add the timing measurements
        training_times.append(t_train_end - t_train_start)
        testing_times.append(t_predict_end - t_predict_start)
        
        timings.loc['train'][i] = t_train_end - t_train_start
        timings.loc['test'][i] = t_predict_end - t_predict_start
        
    timings.loc['total'] = timings.loc['train'] + timings.loc['test']
    timings['mean'] = timings.mean(axis='columns')
    timings['std'] = timings.std(axis='columns')

    return accuracies, timings

def improve_accuracy_output(df_accuracies):
    
    # Move the regular version of the classifier to the top
    original = pd.DataFrame(df_accuracies.loc['None']).T
    df_accuracies = pd.concat([original, df_accuracies.drop('None', axis='index').sort_index(axis='index', ascending=False)])
    df_accuracies.aggregate(['mean', 'min', 'max']).T.reset_index(drop=True)

    df_accuracies['min'] = df_accuracies.min(axis='columns')
    df_accuracies['max'] = df_accuracies.max(axis='columns')
    df_accuracies['mean'] = df_accuracies.mean(axis='columns')
    df_accuracies['mean_diff'] = (df_accuracies['mean'] - df_accuracies.shift(periods=1, axis='index')['mean'])    # Compute how much the mean has changed
    df_accuracies['std'] = df_accuracies.std(axis='columns')
    
    return df_accuracies

## 0.3. Define classifier class

In [6]:
class CustomNaiveBayes:
    """
    Custom version of the Naive Bayes classifier.

    If a Privacy Parameter e is set, this class will be differentially private.
    If no Privacy Parameter is set, the class will act as a regular Naive Bayes classifier.
    """

    # Set the value that should be used instead of zero
    _zero_value = 0.00001

    # Define some class variables that should be remembered between the fitting and prediction methods
    _target = None
    _target_values = None
    _num_features_train = None
    _cat_features_train = None
    _labels_train = None
    _priors = None
    _means = None
    _stds = None
    
    # Add a variable to indicate whether or not the algorithm should output some intermediate steps
    _output = False
    
    # Define whether or not the "enhanced" algorithm should be used.
    # The original implementation is based on the paper by Vaidya, Shafiq, Basu & Hong (see README).
    # The enhanced implementation includes some improvements from the paper by Zafarani & Clifton (see README).
    _enhanced_algorithm = True

    def __init__(self, e=None, enhanced=True):
        """Initialize a new instance of the class."""

        if self._output:
            # Print which Privacy Parameter was set
            self.print_privacy_parameter(e)

        # Set the Privacy Parameter
        self._e = e
        
        self._enhanced_algorithm = enhanced

        return

    @staticmethod
    def print_privacy_parameter(e):
        """Print the Privacy Parameter that was set."""

        if e is None:
            print("No privacy parameter was set")
            print()
        else:
            print("The following privacy parameter was set:", e)
            print()

        return

    @staticmethod
    def laplace(mean, scale):
        """Generate some noise by sampling the Laplace distribution with the given mean and scale."""

        # Sample the Laplace distribution to generate some noise
        return np.random.laplace(loc=mean, scale=scale)

    def get_noise(self, scale_factor):
        """Generate some noise for a given scale factor."""
        
        return self.laplace(0, scale_factor)

    @staticmethod
    def split_data_on_type(features, labels=None):
        """Split the data based on its feature types."""

        # Split features into numerical and categorical features
        num_features = features.select_dtypes(exclude=['object'])
        cat_features = features.select_dtypes(include=['object'])

        # If no labels were specified
        if labels is None:

            # Return just the features
            return num_features, cat_features

        else:

            # Join features and labels to get the full data set
            num_features_incl_labels = num_features.join(labels)
            cat_features_incl_labels = cat_features.join(labels)

            # Return the features AND the labels
            return num_features, cat_features, num_features_incl_labels, cat_features_incl_labels

    def compute_priors(self, training_data, feature):
        """Compute the prior probability for a given feature."""

        # Get the counts for the specified feature
        counts = training_data[feature].value_counts()

        # Add noise to the counts if a privacy parameter was set
        if self._e is not None:
            # Corresponds to LINE 18 of the pseudocode in the original paper
            if self._enhanced_algorithm:
                counts = counts.add(self.laplace(mean=0, scale=(1 / self._e)))
            else:
                counts = counts.add(self.laplace(mean=0, scale=1))

        # Ensure all counts are positive
        counts[counts <= 0] = self._zero_value

        # Compute priors
        priors = counts.div(training_data.shape[0])

        # Ensure all priors are positive
        priors[priors <= 0] = self._zero_value
        
        # Take the logarithm of all priors
        priors = np.log(priors)

        return priors

    def compute_numerical_likelihoods(self, num_training_data, target):
        """Compute the numerical likelihoods for a given target."""

        # Compute the mean per numerical feature and target value
        means = num_training_data.groupby(target).mean()

        # Compute the standard deviation per numerical feature and target value
        stds = num_training_data.groupby(target).std()

        # Add noise to the means and standard deviations if a privacy parameter was set
        if self._e is not None:
            # Compute the noise for the numeric attributes

            # Compute general statistics
            minimums = num_training_data.groupby(target).min()
            maximums = num_training_data.groupby(target).max()

            # Compute the sensitivities for the means and standard deviations
            sensitivities_means = maximums.sub(minimums).div(num_training_data.shape[0] + 1)
            sensitivities_stds = sensitivities_means.mul(np.sqrt(num_training_data.shape[0]))

            # Compute the scale factors
            sf_means = sensitivities_means.div(self._e)
            sf_stds = sensitivities_stds.div(self._e)

            # Compute the noise
            noise_means = sf_means.apply(self.get_noise)
            noise_stds = sf_stds.apply(self.get_noise)

            # Add the noise
            means = means.add(noise_means)  # Corresponds to LINE 10 of the pseudocode in the original paper
            stds = stds.add(noise_stds)  # Corresponds to LINE 13 of the pseudocode in the original paper

        # Ensure all means are positive
        means[means <= 0] = self._zero_value

        # Ensure all standard deviations are positive
        stds[stds <= 0] = self._zero_value

        return means, stds

    def num_cond_probs(self, num_features_y, target_value):
        """This method computes the conditional probabilities for numerical attributes by using the normal probability density function."""

        # Retrieve the feature names from the new instance
        feat_names = num_features_y.columns.values

        # Only consider the relevant feature names for the specified target value
        means = self._means.loc[target_value][feat_names]
        stds = self._stds.loc[target_value][feat_names]

        # Compute the conditional probabilities (returns numpy arrays)
        cond_probs = norm(loc=means.values, scale=stds.values).pdf(num_features_y.values)
        
        # Convert the conditional probabilities to a dataFrame
        cond_probs = pd.DataFrame(cond_probs, columns=feat_names, index=num_features_y.index.values)
          
        cond_probs[cond_probs <= 0] = self._zero_value
        
        # Take the logarithm of the probabilities in order to prevent potential underflows
        log_cond_probs =  np.log(cond_probs)
        
        return log_cond_probs

    def update_privacy_parameter(self, e, cat_features_train, num_features_train):
        """Update the Privacy Parameter for a better representation of the overall privacy."""

        # Don't change the parameter if it has not been set in the first place
        if e is None:
            return None

        if self._output:
            print("Previous value of e:")
            display(e)
            print()
        
        # If the privacy parameter was set, replace it with the overall privacy
        e = e / (1 + cat_features_train.shape[0] + (2 * num_features_train.shape[0]))
        
        if self._output:
            print("New value of e:")
            display(e)
            print()
        
        return e

    def fit(self, features_train, labels_train):
        """Train the classifier with the given features and labels."""

        # Abort if the training labels DataFrame does not contain exactly one column
        if labels_train.shape[1] != 1:
            raise KeyError("The labels DataFrame does not contain exactly one column")

        # Get the target name from the training labels
        target = labels_train.columns.values[0]

        # Abort if the training data contains the target
        if target in features_train.columns.values:
            raise KeyError("The training data should not contain the target")

        # Get the distinct target values
        target_values = labels_train[target].unique()

        # Split features and training data (features incl. labels) into numerical and categorical variables
        num_features_train, cat_features_train, num_training_data, cat_training_data \
            = self.split_data_on_type(features_train, labels_train)

        if self._enhanced_algorithm:
            # Update the privacy parameter if it was set
            self._e = self.update_privacy_parameter(self._e, cat_features_train, num_features_train)

        # Compute the prior probabilities for the target values
        priors = self.compute_priors(labels_train, target)
        
        if self._output:
            print("Priors:")
            display(priors)
            print()
            
        # Only compute numerical likelihoods if the dataset actually contains any numerical features
        if num_features_train.shape[1] > 0:
        
            # Compute the numerical likelihoods for the training data
            means, stds = self.compute_numerical_likelihoods(num_training_data, target)

            if self._output:
                print("Means:")
                display(means)
                print()

                print("Standard Deviations:")
                display(stds)
                print()
                   
            # Store the variables as class variables
            self._means = means
            self._stds = stds  

        # Store the variables as class variables
        self._target = target
        self._target_values = target_values
        self._num_features_train = num_features_train
        self._cat_features_train = cat_features_train
        self._labels_train = labels_train
        self._priors = priors
           
        return

    def predict(self, features_test):
        """Make predictions for the given features."""

        # Reset the indices
        features_test = features_test.reset_index(drop=True)

        # Split testing features into numerical and categorical features
        num_features_test, cat_features_test = self.split_data_on_type(features_test)

        # Retrieve the test features as an array
        test_features = cat_features_test.columns.values

        # Define a results DataFrame that already contains these priors.
        # This DataFrame will later be multiplied with the conditional probability for each feature.
        results = pd.DataFrame([self._priors], columns=self._target_values, index=features_test.index.values)
        
        if self._output:
            print("Results with priors:")
            display(results.head())
            print()

        # Loop over all target values to compute and store the conditional probabilities
        for target_value in self._target_values:

            # Create a target DataFrame that solely consists of the current target value that is being evaluated
            target_labels = pd.DataFrame(target_value, columns=[self._target], index=features_test.index.values)
            
            
            #############
            # NUMERICAL #
            #############

            
            # Only compute numerical conditional probabilities if the training data contains any numerical features
            if self._num_features_train.shape[1] > 0:

                # Compute the intermediate conditional probabilities for all numerical test features, given the target value
                intermediate_cond_probs = self.num_cond_probs(num_features_test, target_value)

                # Compute the sum of the intermediate conditional probabilities and add this to the results.
                # Normally, one would multiply here, but because logarithms are used, we take the sum since log(a*b) = log(a) + log(b)
                results[target_value] += intermediate_cond_probs.sum(axis='columns')

                if self._output:
                    print("Numerical conditional probabilities for label", target_value, ":")
                    display(intermediate_cond_probs.head())
                    print()

                    print("Original data for label", target_value, ":")
                    display(num_features_test.head())
                    print()
               
            
            ###############
            # CATEGORICAL #
            ###############


            # Only compute categorical conditional probabilities if the training data contains any categorical features
            if self._cat_features_train.shape[1] > 0:
                
                # Retrieve the categorical training data
                cat_training_data = self._cat_features_train.join(self._labels_train)

                # Create a subset for which the target values match
                class_matches = cat_training_data.loc[cat_training_data[self._target] == target_value]

                # Define a DataFrame to store the conditional probabilities per target value in
                intermediate_cond_probs = pd.DataFrame()

                # Loop over all the test features
                for feature in test_features:

                    # Compute the value frequencies for each feature
                    value_frequencies = class_matches[feature].value_counts()

                    if self._output:
                        vf = pd.DataFrame(index=['original', 'noisy'], columns=value_frequencies.index.values)
                        vf.loc['original'] = value_frequencies

                    # Add noise to the counts if a privacy parameter was set
                    if self._e is not None:
                        # Corresponds to LINE 5 of the pseudocode in the original paper
                        value_frequencies = value_frequencies.add(self.laplace(mean=0, scale=(1 / self._e)))

                    # Ensure the (noisy) value frequencies are positive
                    value_frequencies[value_frequencies <= 0] = self._zero_value

                    if self._output:    
                        vf.loc['noisy'] = value_frequencies

                        print("Value frequencies for label", target_value, ":")
                        display(vf)
                        print()

                    # Compute the number of records of class matches
                    n_class_matches = class_matches.shape[0]

                    if n_class_matches <= 0:
                        n_class_matches = self._zero_value

                    # Compute the conditional probabilities for the categorical feature
                    #    by considering each feature value, getting its frequency of occurrence
                    #    and dividing it by the total number of records for which the target value matches
                    cond_probs = value_frequencies[self._cat_features_train[feature]] / n_class_matches

                    # Store the conditional probabilities
                    intermediate_cond_probs[feature] = np.log(cond_probs.values)


                if self._output:
                    print("Categorical conditional probabilities for label", target_value, ":")
                    display(intermediate_cond_probs.head())
                    print()

                    print("Original data for label", target_value, ":")
                    display(cat_features_test.head())
                    print()

                # Compute the sum of the intermediate conditional probabilities and add this to the results.
                # Normally, one would multiply here, but because logarithms are used, we take the sum since log(a*b) = log(a) + log(b)
                results[target_value] += intermediate_cond_probs.sum(axis='columns')

        # Make predictions by takin the column with the highest score as prediction
        predictions = results.idxmax(axis='columns')

        # Return the predictions
        return predictions
    
    def enable_output(self):
        """Enable intermediate output for the classifier."""
        
        self._output = True
        
    def disable_output(self):
        """Disable intermediate output for the classifier."""
        
        self._output = False

In [7]:
def run_classifier(dataset, target, buckets_loc):
    """Run the classifier on a given dataset, for a specific target."""
    
    # Define a list of privacy parameters for which the results should be generated
    privacy_parameters = [None, 1.0, 0.75, 0.5, 0.25, 0.1, 0.05, 0.01, 0.005, 0.001, 0.0000000001]
    privacy_parameters_without_None = privacy_parameters[-(len(privacy_parameters) - 1):]

    # Define which headings to use for the accuracy DataFrames
    accuracy_headings = ['accuracy 1', 'accuracy 2', 'accuracy 3', 'accuracy 4', 'accuracy 5', 'accuracy 6', 'accuracy 7', 'accuracy 8', 'accuracy 9', 'accuracy 10']

    # Define a DataFrame to store the computed accuracies in
    df_accuracies_original = pd.DataFrame([], columns=accuracy_headings, index=privacy_parameters_without_None)
    df_accuracies_enhanced = pd.DataFrame([], columns=accuracy_headings, index=privacy_parameters_without_None)

    # Run the classifier for each epsilon value
    for e in privacy_parameters:
        
        if e is None:
            index = 'None'
        else:
            index = e

        # Generate results for both the original and the enhanced implementation.
        # The original implementation is based on the paper by Vaidya, Shafiq, Basu & Hong (see README).
        # The enhanced implementation includes some improvements from the paper by Zafarani & Clifton (see README).
        df_accuracies_original.loc[index], timings_original = cross_validation(dataset, target, buckets_loc, e, enhanced=False)
        df_accuracies_enhanced.loc[index], timings_enhanced = cross_validation(dataset, target, buckets_loc, e, enhanced=True)

        print("Timings (in seconds) for original algorithm with e =", e, ":")
        display(timings_original)
        print()

        print("Timings (in seconds) for enhanced algorithm with e =", e, ":")
        display(timings_enhanced)
        print()


    print("Original Algorithm:")
    display(improve_accuracy_output(df_accuracies_original))
    print()

    print("Enhanced Algorithm:")
    display(improve_accuracy_output(df_accuracies_enhanced))
    print()
    
    return

# Adult Dataset

In [8]:
%%time

# Define the headings to be used
headings = ['age', 'workclass', 'final-weight', 'education', 'education-num', 'marital-status', 'occupation',
            'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week', 'native-country',
            'income']

# Read in the official training and testing data separately
data_train = pd.read_csv('datasets/Adult/adult.data', names=headings)
data_test = pd.read_csv('datasets/Adult/adult.test', names=headings, skiprows=1)

# Combine both data sets into one data set
adult_data = data_train.append(data_test)

# Apply some pre-processing
adult_data['age'] = adult_data['age'].astype('int').apply(pd.to_numeric, downcast="unsigned")
adult_data['workclass'] = adult_data['workclass'].astype('category').str.strip()
adult_data['final-weight'] = adult_data['final-weight'].astype('int').apply(pd.to_numeric, downcast="unsigned")
adult_data['education'] = adult_data['education'].astype('category').str.strip()
adult_data['education-num'] = adult_data['education-num'].astype('int').apply(pd.to_numeric, downcast="unsigned")
adult_data['marital-status'] = adult_data['marital-status'].astype('category').str.strip()
adult_data['occupation'] = adult_data['occupation'].astype('category').str.strip()
adult_data['relationship'] = adult_data['relationship'].astype('category').str.strip()
adult_data['race'] = adult_data['race'].astype('category').str.strip()
adult_data['sex'] = adult_data['sex'].astype('category').str.strip()
adult_data['capital-gain'] = adult_data['capital-gain'].astype('int').apply(pd.to_numeric, downcast="unsigned")
adult_data['capital-loss'] = adult_data['capital-loss'].astype('int').apply(pd.to_numeric, downcast="unsigned")
adult_data['hours-per-week'] = adult_data['hours-per-week'].astype('int').apply(pd.to_numeric, downcast="unsigned")
adult_data['native-country'] = adult_data['native-country'].astype('category').str.strip()
adult_data['income'] = adult_data['income'].astype('category').str.strip()

# Remove the dot from the income column
adult_data['income'] = adult_data['income'].replace({'\.': ''}, regex=True)

# Convert every '?' to NaN, so Pandas' built-in functions can be used
adult_data = adult_data.replace({'?': np.NaN})

# Try to fill missing values with the mean (this only works for numerical attributes)
adult_data = adult_data.fillna(adult_data.mean())

# Get the number of records in the data set
n_records_before = adult_data.shape[0]

# If values are still missing (they must be categorical attributes), drop the rows with missing data
adult_data = adult_data.dropna()

# Get the number of records in the data set
n_records_after = adult_data.shape[0]

# Print how many records containing NaN values got dropped
n_records_dropped = n_records_before - n_records_after
print(n_records_dropped, "records were dropped due to missing values.")
print("This is", round(n_records_dropped / n_records_before * 100, 1), "% of the entire data set.")
print("The resulting data set contains", n_records_after, "records.")

# Reset the indices
adult_data = adult_data.reset_index(drop=True)

# Print a newline to separate outputs
print()

3620 records were dropped due to missing values.
This is 7.4 % of the entire data set.
The resulting data set contains 45222 records.

Wall time: 20.5 s


In [9]:
display(adult_data.head())

adult_data.shape

,age,workclass,final-weight,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


(45222, 15)

In [10]:
%%time

run_classifier(adult_data, 'income', 'datasets/Adult/buckets_adult.csv')

Timings (in seconds) for original algorithm with e = None :


,fold 1,fold 2,fold 3,fold 4,fold 5,fold 6,fold 7,fold 8,fold 9,fold 10,mean,std
train,0.059963,0.0479681,0.0459874,0.0479755,0.0479705,0.047971,0.0469732,0.0539668,0.0472512,0.0469718,0.049300,0.004105
test,0.266015,0.272833,0.271266,0.269088,0.268094,0.265876,0.266286,0.268834,0.27105,0.277131,0.269647,0.003359
total,0.325978,0.320801,0.317253,0.317064,0.316064,0.313847,0.313259,0.322801,0.318301,0.324103,0.318947,0.004097



Timings (in seconds) for enhanced algorithm with e = None :


,fold 1,fold 2,fold 3,fold 4,fold 5,fold 6,fold 7,fold 8,fold 9,fold 10,mean,std
train,0.0489681,0.0473366,0.0470426,0.0509682,0.0449715,0.0449739,0.0469708,0.0459719,0.0489712,0.049969,0.047614,0.001948
test,0.280245,0.2701,0.2678,0.279261,0.265165,0.269832,0.265841,0.264836,0.270101,0.271161,0.270434,0.005123
total,0.329213,0.317436,0.314842,0.330229,0.310136,0.314806,0.312812,0.310807,0.319072,0.32113,0.318048,0.006687



Timings (in seconds) for original algorithm with e = 1.0 :


...


Original Algorithm:


,accuracy 1,accuracy 2,accuracy 3,accuracy 4,accuracy 5,accuracy 6,accuracy 7,accuracy 8,accuracy 9,accuracy 10,min,max,mean,mean_diff,std
None,0.778859,0.781955,0.780849,0.776648,0.774878,0.783724,0.784830,0.778638,0.775763,0.768789,0.768789,0.784830,0.778213,NaN,0.005297
1.0,0.779080,0.780849,0.782839,0.776426,0.776426,0.769129,0.785493,0.764042,0.775984,0.765694,0.764042,0.785493,0.775458,-0.002754,0.208115
0.75,0.778417,0.782618,0.781955,0.774215,0.769129,0.782176,0.784609,0.779080,0.770898,0.750663,0.750663,0.784609,0.774086,-0.001372,0.207546
0.5,0.765148,0.756966,0.755639,0.782176,0.776869,0.663644,0.661433,0.779965,0.774878,0.765915,0.661433,0.782176,0.743854,-0.030232,0.211946
0.25,0.770456,0.784166,0.768023,0.601061,0.601725,0.766254,0.780407,0.780628,0.664087,0.745137,0.601061,0.784166,0.720598,-0.023256,0.211657
0.1,0.773552,0.762716,0.239938,0.754091,0.747899,0.607696,0.557497,0.666962,0.705882,0.755747,0.239938,0.773552,0.632122,-0.088475,0.263589
0.05,0.753870,0.684432,0.430119,0.719814,0.623397,0.370854,0.731314,0.590668,0.705440,0.684571,0.370854,0.753870,0.618267,-0.013856,0.215887
0.01,0.686643,0.755418,0.417072,0.743476,0.685980,0.686201,0.249005,0.483193,0.409332,0.732317,0.249005,0.755418,0.571088,-0.047179,0.245499
0.005,0.720035,0.454003,0.291243,0.367315,0.756524,0.516143,0.510615,0.769571,0.749226,0.471264,0.291243,0.769571,0.555563,-0.015526,0.231309
0.001,0.474569,0.319328,0.609244,0.431889,0.735073,0.628925,0.708757,0.743255,0.639982,0.532935,0.319328,0.743255,0.573878,0.018316,0.207197



Enhanced Algorithm:


,accuracy 1,accuracy 2,accuracy 3,accuracy 4,accuracy 5,accuracy 6,accuracy 7,accuracy 8,accuracy 9,accuracy 10,min,max,mean,mean_diff,std
None,0.778859,0.781955,0.780849,0.776648,0.774878,0.783724,0.784830,0.778638,0.775763,0.768789,0.768789,0.784830,0.778213,NaN,0.005297
1.0,0.753207,0.241486,0.753428,0.751880,0.749668,0.234852,0.752322,0.230650,0.750774,0.259726,0.230650,0.753428,0.538506,-0.239707,0.319786
0.75,0.247678,0.758514,0.246572,0.248120,0.250332,0.251437,0.247678,0.769350,0.250774,0.736295,0.246572,0.769350,0.418556,-0.119950,0.272157
0.5,0.247678,0.241486,0.753428,0.248120,0.250332,0.251437,0.247678,0.769350,0.250774,0.740274,0.241486,0.769350,0.417616,-0.000940,0.257047
0.25,0.753207,0.758514,0.246572,0.226891,0.250332,0.227996,0.752322,0.769350,0.749226,0.595712,0.226891,0.769350,0.527197,0.109581,0.265123
0.1,0.247678,0.241486,0.246572,0.248120,0.749668,0.227996,0.247678,0.230650,0.250774,0.736295,0.227996,0.749668,0.367049,-0.160148,0.252821
0.05,0.752322,0.241486,0.753428,0.751880,0.775984,0.748341,0.752322,0.230650,0.749226,0.736295,0.230650,0.775984,0.624881,0.257832,0.238077
0.01,0.752322,0.241486,0.753428,0.248120,0.749668,0.748563,0.752322,0.230650,0.250774,0.263705,0.230650,0.753428,0.497926,-0.126954,0.295577
0.005,0.752322,0.241486,0.246572,0.248120,0.250332,0.251437,0.247678,0.230650,0.768686,0.736295,0.230650,0.768686,0.414410,-0.083517,0.267981
0.001,0.752322,0.241486,0.222689,0.750995,0.250332,0.748563,0.752322,0.769350,0.250774,0.263705,0.222689,0.769350,0.499548,0.085138,0.271406



Wall time: 2min 13s


# Mushroom Dataset

In [11]:
%%time

# Define the headings to be used
headings = ['label', 'cap-shape', 'cap-surface', 'cap-color', 'bruises', 'odor', 'gill-attachment', 'gill-spacing', 'gill-size', 'gill-color', 'stalk-shape', 'stalk-root', 'stalk-surface-above-ring', 'stalk-surface-below-ring', 'stalk-color-above-ring', 'stalk-color-below-ring', 'veil-type', 'veil-color', 'ring-number', 'ring-type', 'spore-print-color', 'population', 'habitat']

# Read in the official training and testing data separately
mushroom_data = pd.read_csv('datasets/Mushroom/agaricus-lepiota.data', names=headings)

# Apply some pre-processing
for heading in headings:
    mushroom_data[heading] = mushroom_data[heading].astype('category').str.strip()
    
# Convert every '?' to NaN, so Pandas' built-in functions can be used
mushroom_data = mushroom_data.replace({'?': np.NaN})

# Find out which features contain missing (NaN) values
nan_features = mushroom_data.columns[mushroom_data.isnull().any()]
nan_features_count = mushroom_data[nan_features].isnull().sum()

# According to the following overview, all missing values occur in the 'stalk-root' feature, so let's drop this feature.
# The alternative would be to drop all rows that contain a missing value, but this would be 30.5% of the entire dataset, which is a bit much.

print("Number of records with NaN values, per feature:")
display(nan_features_count)
print()

# Drop the 'stalk-root' feature
mushroom_data = mushroom_data.drop('stalk-root', axis='columns')

# Get the number of records in the data set
n_records_before = mushroom_data.shape[0]

# If values are still missing (they must be categorical attributes), drop the rows with missing data
mushroom_data = mushroom_data.dropna()

# Get the number of records in the data set
n_records_after = mushroom_data.shape[0]

# Print how many records containing NaN values got dropped
n_records_dropped = n_records_before - n_records_after
print(n_records_dropped, "records were dropped due to missing values.")
print("This is", round(n_records_dropped / n_records_before * 100, 1), "% of the entire data set.")
print("The resulting data set contains", n_records_after, "records.")

# Reset the indices
mushroom_data = mushroom_data.reset_index(drop=True)

# Print a newline to separate outputs
print()

Number of records with NaN values, per feature:


stalk-root    2480
dtype: int64


0 records were dropped due to missing values.
This is 0.0 % of the entire data set.
The resulting data set contains 8124 records.

Wall time: 110 ms


In [12]:
display(mushroom_data.head())

mushroom_data.shape

,label,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,p,x,s,n,t,p,f,c,n,k,...,s,w,w,p,w,o,p,k,s,u
1,e,x,s,y,t,a,f,c,b,k,...,s,w,w,p,w,o,p,n,n,g
2,e,b,s,w,t,l,f,c,b,n,...,s,w,w,p,w,o,p,n,n,m
3,p,x,y,w,t,p,f,c,n,n,...,s,w,w,p,w,o,p,k,s,u
4,e,x,s,g,f,n,f,w,b,k,...,s,w,w,p,w,o,e,n,a,g


(8124, 22)

In [13]:
%%time

run_classifier(mushroom_data, 'label', 'datasets/Mushroom/buckets_mushroom.csv')

Timings (in seconds) for original algorithm with e = None :


,fold 1,fold 2,fold 3,fold 4,fold 5,fold 6,fold 7,fold 8,fold 9,fold 10,mean,std
train,0.010993,0.0109937,0.010993,0.0109932,0.0119925,0.010994,0.0109932,0.0109937,0.0109935,0.0109937,0.011093,0.000300
test,0.201109,0.199877,0.197878,0.202874,0.198877,0.201943,0.199875,0.200094,0.203948,0.200876,0.200735,0.001733
total,0.212102,0.21087,0.208871,0.213868,0.21087,0.212937,0.210868,0.211088,0.214942,0.21187,0.211828,0.001650



Timings (in seconds) for enhanced algorithm with e = None :


,fold 1,fold 2,fold 3,fold 4,fold 5,fold 6,fold 7,fold 8,fold 9,fold 10,mean,std
train,0.0119948,0.0109932,0.0159903,0.0119874,0.0109937,0.0109935,0.0110531,0.0109928,0.0109928,0.0109932,0.011698,0.001483
test,0.198875,0.200686,0.203539,0.209871,0.202162,0.199877,0.210811,0.206875,0.198295,0.198388,0.202938,0.004469
total,0.21087,0.211679,0.21953,0.221858,0.213156,0.21087,0.221864,0.217867,0.209287,0.209381,0.214636,0.004841



Timings (in seconds) for original algorithm with e = 1.0 :


...


Original Algorithm:


,accuracy 1,accuracy 2,accuracy 3,accuracy 4,accuracy 5,accuracy 6,accuracy 7,accuracy 8,accuracy 9,accuracy 10,min,max,mean,mean_diff,std
None,0.512315,0.508621,0.514778,0.538177,0.500000,0.508621,0.523399,0.533251,0.512315,0.518382,0.500000,0.538177,0.517336,NaN,0.012812
1.0,0.512315,0.508621,0.516010,0.538177,0.500000,0.508621,0.523399,0.533251,0.512315,0.518382,0.500000,0.538177,0.517439,0.000103,0.138809
0.75,0.512315,0.508621,0.514778,0.538177,0.500000,0.508621,0.523399,0.533251,0.512315,0.518382,0.500000,0.538177,0.517336,-0.000103,0.138838
0.5,0.512315,0.508621,0.517241,0.538177,0.500000,0.508621,0.523399,0.533251,0.512315,0.518382,0.500000,0.538177,0.517542,0.000205,0.138809
0.25,0.512315,0.509852,0.514778,0.538177,0.500000,0.508621,0.522167,0.533251,0.513547,0.518382,0.500000,0.538177,0.517439,-0.000103,0.138853
0.1,0.511084,0.509852,0.514778,0.539409,0.507389,0.514778,0.522167,0.533251,0.508621,0.520833,0.507389,0.539409,0.519080,0.001641,0.138738
0.05,0.520936,0.504926,0.514778,0.538177,0.506158,0.509852,0.525862,0.534483,0.508621,0.523284,0.504926,0.538177,0.519182,0.000102,0.139232
0.01,0.508621,0.511084,0.518473,0.539409,0.503695,0.516010,0.525862,0.524631,0.514778,0.523284,0.503695,0.539409,0.519079,-0.000103,0.139199
0.005,0.500000,0.513547,0.514778,0.536946,0.524631,0.507389,0.507389,0.498768,0.504926,0.515931,0.498768,0.536946,0.513335,-0.005744,0.139282
0.001,0.513547,0.492611,0.514778,0.525862,0.514778,0.500000,0.525862,0.527094,0.513547,0.522059,0.492611,0.527094,0.514154,0.000819,0.137699



Enhanced Algorithm:


,accuracy 1,accuracy 2,accuracy 3,accuracy 4,accuracy 5,accuracy 6,accuracy 7,accuracy 8,accuracy 9,accuracy 10,min,max,mean,mean_diff,std
None,0.512315,0.508621,0.514778,0.538177,0.500000,0.508621,0.523399,0.533251,0.512315,0.518382,0.500000,0.538177,0.517336,NaN,0.012812
1.0,0.501232,0.507389,0.507389,0.460591,0.492611,0.491379,0.525862,0.471675,0.486453,0.476716,0.460591,0.525862,0.492313,-0.025024,0.139786
0.75,0.487685,0.492611,0.492611,0.539409,0.492611,0.516010,0.525862,0.528325,0.486453,0.477941,0.477941,0.539409,0.504739,0.012426,0.133329
0.5,0.512315,0.492611,0.524631,0.536946,0.492611,0.483990,0.525862,0.528325,0.513547,0.477941,0.477941,0.536946,0.508639,0.003900,0.136481
0.25,0.512315,0.507389,0.507389,0.461823,0.492611,0.516010,0.495074,0.471675,0.513547,0.477941,0.461823,0.516010,0.494467,-0.014172,0.137311
0.1,0.487685,0.490148,0.492611,0.460591,0.507389,0.516010,0.474138,0.498768,0.486453,0.477941,0.460591,0.516010,0.489028,-0.005439,0.133274
0.05,0.487685,0.525862,0.497537,0.516010,0.492611,0.483990,0.470443,0.471675,0.486453,0.477941,0.470443,0.525862,0.492209,0.003181,0.132032
0.01,0.487685,0.492611,0.525862,0.502463,0.507389,0.483990,0.520936,0.471675,0.513547,0.500000,0.471675,0.525862,0.500308,0.008098,0.132738
0.005,0.487685,0.507389,0.507389,0.522167,0.507389,0.516010,0.474138,0.528325,0.516010,0.509804,0.474138,0.528325,0.506564,0.006256,0.134813
0.001,0.483990,0.507389,0.507389,0.523399,0.492611,0.516010,0.527094,0.528325,0.524631,0.477941,0.477941,0.528325,0.507920,0.001356,0.136626



Wall time: 1min 1s


# Congressional Voting Dataset

In [14]:
%%time

# Define the headings to be used
headings = ['party', 'handicapped-infants', 'water-project-cost-sharing', 'adoption-of-the-budget-resolution', 'physician-fee-freeze', 'el-salvador-aid', 'religious-groups-in-schools', 'anti-satellite-test-ban', 'aid-to-nicaraguan-contras', 'mx-missile', 'immigration', 'synfuels-corporation-cutback', 'education-spending', 'superfund-right-to-sue', 'crime', 'duty-free-exports', 'export-administration-act-south-africa']

# Read in the official training and testing data separately
congressional_votes = pd.read_csv('datasets/Congressional_Voting_Records/house-votes-84.data', names=headings)

# Apply some pre-processing
for heading in headings:
    congressional_votes[heading] = congressional_votes[heading].astype('category').str.strip()
    
# Convert every '?' to NaN, so Pandas' built-in functions can be used
congressional_votes = congressional_votes.replace({'?': np.NaN})

# Find out which features contain missing (NaN) values
nan_features = congressional_votes.columns[congressional_votes.isnull().any()]
nan_features_count = congressional_votes[nan_features].isnull().sum()

print("Number of records with NaN values, per feature:")
display(nan_features_count)
print()

# Get the number of records in the data set
n_records_before = congressional_votes.shape[0]

# Drop the rows with missing values, since they occur in multiple features
congressional_votes = congressional_votes.dropna()

# Get the number of records in the data set
n_records_after = congressional_votes.shape[0]

# Print how many records containing NaN values got dropped
n_records_dropped = n_records_before - n_records_after
print(n_records_dropped, "records were dropped due to missing values.")
print("This is", round(n_records_dropped / n_records_before * 100, 1), "% of the entire data set.")
print("The resulting data set contains", n_records_after, "records.")

# Reset the indices
congressional_votes = congressional_votes.reset_index(drop=True)

# Print a newline to separate outputs
print()

Number of records with NaN values, per feature:


handicapped-infants                        12
water-project-cost-sharing                 48
adoption-of-the-budget-resolution          11
physician-fee-freeze                       11
el-salvador-aid                            15
religious-groups-in-schools                11
anti-satellite-test-ban                    14
aid-to-nicaraguan-contras                  15
mx-missile                                 22
immigration                                 7
synfuels-corporation-cutback               21
education-spending                         31
superfund-right-to-sue                     25
crime                                      17
duty-free-exports                          28
export-administration-act-south-africa    104
dtype: int64


203 records were dropped due to missing values.
This is 46.7 % of the entire data set.
The resulting data set contains 232 records.

Wall time: 34 ms


In [15]:
display(congressional_votes.head())

congressional_votes.shape

,party,handicapped-infants,water-project-cost-sharing,adoption-of-the-budget-resolution,physician-fee-freeze,el-salvador-aid,religious-groups-in-schools,anti-satellite-test-ban,aid-to-nicaraguan-contras,mx-missile,immigration,synfuels-corporation-cutback,education-spending,superfund-right-to-sue,crime,duty-free-exports,export-administration-act-south-africa
0,democrat,n,y,y,n,y,y,n,n,n,n,n,n,y,y,y,y
1,republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,y
2,democrat,y,y,y,n,n,n,y,y,y,n,y,n,n,n,y,y
3,democrat,y,y,y,n,n,n,y,y,y,n,n,n,n,n,y,y
4,democrat,y,n,y,n,n,n,y,y,y,y,n,n,n,n,y,y


(232, 17)

In [16]:
%%time

run_classifier(congressional_votes, 'party', 'datasets/Congressional_Voting_Records/buckets_congressional_voting.csv')

Timings (in seconds) for original algorithm with e = None :


,fold 1,fold 2,fold 3,fold 4,fold 5,fold 6,fold 7,fold 8,fold 9,fold 10,mean,std
train,0.00699425,0.0059967,0.00699615,0.00699782,0.0059936,0.0059948,0.00599837,0.00599718,0.00599742,0.00699449,0.006396,0.000490
test,0.108935,0.111932,0.100175,0.0999386,0.0989392,0.100938,0.0999367,0.0989399,0.103935,0.0999897,0.102366,0.004300
total,0.115929,0.117928,0.107172,0.106936,0.104933,0.106932,0.105935,0.104937,0.109932,0.106984,0.108762,0.004318



Timings (in seconds) for enhanced algorithm with e = None :


,fold 1,fold 2,fold 3,fold 4,fold 5,fold 6,fold 7,fold 8,fold 9,fold 10,mean,std
train,0.00599623,0.00699759,0.0069952,0.00599813,0.00699425,0.00599766,0.00599766,0.00599432,0.00699401,0.00699568,0.006496,0.000499
test,0.100177,0.102936,0.0999393,0.100935,0.0979402,0.0999367,0.0999379,0.0989418,0.0989397,0.0982685,0.099795,0.001364
total,0.106173,0.109934,0.106935,0.106933,0.104934,0.105934,0.105936,0.104936,0.105934,0.105264,0.106291,0.001386



Timings (in seconds) for original algorithm with e = 1.0 :


...


Original Algorithm:


,accuracy 1,accuracy 2,accuracy 3,accuracy 4,accuracy 5,accuracy 6,accuracy 7,accuracy 8,accuracy 9,accuracy 10,min,max,mean,mean_diff,std
None,0.434783,0.304348,0.521739,0.565217,0.565217,0.347826,0.478261,0.478261,0.565217,0.60,0.304348,0.600000,0.480435,NaN,0.105496
1.0,0.434783,0.304348,0.521739,0.565217,0.565217,0.304348,0.434783,0.478261,0.565217,0.60,0.304348,0.600000,0.473188,-0.007246,0.167046
0.75,0.434783,0.304348,0.521739,0.521739,0.565217,0.347826,0.478261,0.478261,0.565217,0.64,0.304348,0.640000,0.483478,0.010290,0.165995
0.5,0.434783,0.304348,0.521739,0.565217,0.565217,0.347826,0.478261,0.521739,0.565217,0.60,0.304348,0.600000,0.484058,0.000580,0.164586
0.25,0.434783,0.304348,0.521739,0.521739,0.565217,0.304348,0.478261,0.478261,0.565217,0.60,0.304348,0.600000,0.473188,-0.010870,0.166049
0.1,0.434783,0.347826,0.521739,0.478261,0.565217,0.347826,0.478261,0.521739,0.521739,0.60,0.347826,0.600000,0.480435,0.007246,0.152835
0.05,0.434783,0.304348,0.565217,0.521739,0.521739,0.347826,0.391304,0.521739,0.478261,0.60,0.304348,0.600000,0.465942,-0.014493,0.162277
0.01,0.347826,0.217391,0.521739,0.521739,0.304348,0.347826,0.434783,0.521739,0.478261,0.56,0.217391,0.560000,0.419420,-0.046522,0.171274
0.005,0.521739,0.478261,0.391304,0.652174,0.304348,0.304348,0.434783,0.608696,0.565217,0.56,0.304348,0.652174,0.481449,0.062029,0.165728
0.001,0.391304,0.478261,0.391304,0.347826,0.695652,0.608696,0.260870,0.521739,0.521739,0.52,0.260870,0.695652,0.474493,-0.006957,0.187857



Enhanced Algorithm:


,accuracy 1,accuracy 2,accuracy 3,accuracy 4,accuracy 5,accuracy 6,accuracy 7,accuracy 8,accuracy 9,accuracy 10,min,max,mean,mean_diff,std
None,0.434783,0.304348,0.521739,0.565217,0.565217,0.347826,0.478261,0.478261,0.565217,0.60,0.304348,0.600000,0.480435,NaN,0.105496
1.0,0.608696,0.478261,0.608696,0.347826,0.304348,0.391304,0.739130,0.608696,0.478261,0.48,0.304348,0.739130,0.507391,0.026957,0.191424
0.75,0.521739,0.304348,0.608696,0.347826,0.304348,0.608696,0.260870,0.608696,0.478261,0.52,0.260870,0.608696,0.452754,-0.054638,0.191142
0.5,0.391304,0.521739,0.608696,0.347826,0.695652,0.608696,0.260870,0.608696,0.478261,0.48,0.260870,0.695652,0.496522,0.043768,0.186648
0.25,0.608696,0.521739,0.608696,0.652174,0.304348,0.391304,0.739130,0.391304,0.478261,0.56,0.304348,0.739130,0.524928,0.028406,0.193620
0.1,0.391304,0.521739,0.391304,0.347826,0.695652,0.608696,0.260870,0.391304,0.478261,0.48,0.260870,0.695652,0.460290,-0.064638,0.196044
0.05,0.391304,0.521739,0.391304,0.652174,0.304348,0.608696,0.260870,0.608696,0.478261,0.52,0.260870,0.652174,0.470870,0.010580,0.182739
0.01,0.608696,0.521739,0.391304,0.652174,0.304348,0.391304,0.739130,0.608696,0.521739,0.52,0.304348,0.739130,0.525217,0.054348,0.188300
0.005,0.608696,0.478261,0.608696,0.652174,0.304348,0.391304,0.260870,0.608696,0.478261,0.48,0.260870,0.652174,0.482029,-0.043188,0.196084
0.001,0.608696,0.478261,0.608696,0.347826,0.304348,0.391304,0.260870,0.391304,0.521739,0.52,0.260870,0.608696,0.441884,-0.040145,0.177904



Wall time: 27.6 s


# Nursery Dataset

In [17]:
%%time

# Define the headings to be used
headings = ['parents', 'has_nurs', 'form', 'children', 'housing', 'finance', 'social', 'health']

# Read in the official training and testing data separately
nursery_data = pd.read_csv('datasets/Nursery/nursery.data', names=headings)

# Apply some pre-processing
for heading in headings:
    nursery_data[heading] = nursery_data[heading].astype('category').str.strip()
    
# Convert every '?' to NaN, so Pandas' built-in functions can be used
nursery_data = nursery_data.replace({'?': np.NaN})

# Find out which features contain missing (NaN) values
nan_features = nursery_data.columns[nursery_data.isnull().any()]
nan_features_count = nursery_data[nan_features].isnull().sum()

print("Number of records with NaN values, per feature:")
display(nan_features_count)
print()

# Get the number of records in the data set
n_records_before = nursery_data.shape[0]

# Drop the rows with missing values, since they occur in multiple features
nursery_data = nursery_data.dropna()

# Get the number of records in the data set
n_records_after = nursery_data.shape[0]

# Print how many records containing NaN values got dropped
n_records_dropped = n_records_before - n_records_after
print(n_records_dropped, "records were dropped due to missing values.")
print("This is", round(n_records_dropped / n_records_before * 100, 1), "% of the entire data set.")
print("The resulting data set contains", n_records_after, "records.")

# Reset the indices
nursery_data = nursery_data.reset_index(drop=True)

# Print a newline to separate outputs
print()

Number of records with NaN values, per feature:


Series([], dtype: float64)


0 records were dropped due to missing values.
This is 0.0 % of the entire data set.
The resulting data set contains 12960 records.

Wall time: 93.5 ms


In [18]:
display(nursery_data.head())

nursery_data.shape

,parents,has_nurs,form,children,housing,finance,social,health
0,proper,complete,1,convenient,convenient,nonprob,recommended,recommend
1,proper,complete,1,convenient,convenient,nonprob,priority,priority
2,proper,complete,1,convenient,convenient,nonprob,not_recom,not_recom
3,proper,complete,1,convenient,convenient,slightly_prob,recommended,recommend
4,proper,complete,1,convenient,convenient,slightly_prob,priority,priority


(12960, 8)

In [19]:
%%time

run_classifier(nursery_data, 'health', 'datasets/Nursery/buckets_nursery.csv')

Timings (in seconds) for original algorithm with e = None :


,fold 1,fold 2,fold 3,fold 4,fold 5,fold 6,fold 7,fold 8,fold 9,fold 10,mean,std
train,0.0169892,0.00999236,0.00999379,0.00899267,0.00999403,0.0119934,0.00999403,0.0119917,0.0109925,0.0109928,0.011193,0.002134
test,0.205872,0.197211,0.198489,0.194766,0.198222,0.239851,0.201875,0.278066,0.288939,0.238043,0.224133,0.033607
total,0.222862,0.207203,0.208483,0.203759,0.208216,0.251844,0.211869,0.290058,0.299931,0.249036,0.235326,0.034027



Timings (in seconds) for enhanced algorithm with e = None :


,fold 1,fold 2,fold 3,fold 4,fold 5,fold 6,fold 7,fold 8,fold 9,fold 10,mean,std
train,0.00999379,0.0139918,0.0179899,0.0199881,0.00999379,0.0109937,0.0139923,0.0159898,0.0129926,0.0119934,0.013792,0.003186
test,0.230092,0.251846,0.335793,0.314806,0.237855,0.22309,0.22486,0.300816,0.237181,0.268836,0.262518,0.038727
total,0.240086,0.265838,0.353783,0.334795,0.247849,0.234084,0.238853,0.316806,0.250174,0.280829,0.276310,0.041453



Timings (in seconds) for original algorithm with e = 1.0 :


...


Original Algorithm:


,accuracy 1,accuracy 2,accuracy 3,accuracy 4,accuracy 5,accuracy 6,accuracy 7,accuracy 8,accuracy 9,accuracy 10,min,max,mean,mean_diff,std
None,0.318673,0.298611,0.312500,0.307099,0.311728,0.324846,0.298611,0.332562,0.328704,0.314815,0.298611,0.332562,0.314943,NaN,0.012248
1.0,0.239198,0.295525,0.309414,0.131173,0.260031,0.229167,0.248457,0.289352,0.290123,0.260802,0.131173,0.309414,0.249486,-0.065458,0.101300
0.75,0.205247,0.300154,0.161265,0.310957,0.165895,0.283179,0.277006,0.031636,0.327932,0.292438,0.031636,0.327932,0.226273,-0.023212,0.119411
0.5,0.212963,0.292438,0.256173,0.307099,0.267747,0.327160,0.145833,0.334877,0.079475,0.214506,0.079475,0.334877,0.237719,0.011445,0.104935
0.25,0.288580,0.089506,0.312500,0.283179,0.273148,0.016975,0.259259,0.310185,0.334877,0.264660,0.016975,0.334877,0.232060,-0.005658,0.126592
0.1,0.320216,0.125000,0.281636,0.212191,0.277778,0.321759,0.287037,0.287809,0.315586,0.304012,0.125000,0.321759,0.264982,0.032922,0.090729
0.05,0.321759,0.232253,0.306327,0.304784,0.249228,0.175154,0.249228,0.340278,0.290123,0.288580,0.175154,0.340278,0.272762,0.007780,0.088241
0.01,0.327160,0.326389,0.236883,0.021605,0.064043,0.263889,0.307099,0.239969,0.287037,0.290895,0.021605,0.327160,0.226145,-0.046618,0.131757
0.005,0.017747,0.263117,0.290123,0.111111,0.293210,0.324846,0.277778,0.005401,0.273920,0.259259,0.005401,0.324846,0.203897,-0.022248,0.133345
0.001,0.326389,0.141975,0.145833,0.069444,0.155864,0.300154,0.100309,0.108796,0.306327,0.111111,0.069444,0.326389,0.180170,-0.023727,0.109427



Enhanced Algorithm:


,accuracy 1,accuracy 2,accuracy 3,accuracy 4,accuracy 5,accuracy 6,accuracy 7,accuracy 8,accuracy 9,accuracy 10,min,max,mean,mean_diff,std
None,0.318673,0.298611,0.312500,0.307099,0.311728,0.324846,0.298611,0.332562,0.328704,0.314815,0.298611,0.332562,0.314943,NaN,0.012248
1.0,0.016204,0.002315,0.002315,0.003086,0.163580,0.153549,0.018519,0.241512,0.010031,0.016204,0.002315,0.241512,0.072595,-0.242348,0.123054
0.75,0.026235,0.003086,0.058642,0.152778,0.278549,0.216049,0.004630,0.218364,0.017747,0.006173,0.003086,0.278549,0.105324,0.032729,0.107122
0.5,0.029321,0.002315,0.037037,0.026235,0.003086,0.074846,0.020833,0.020062,0.017747,0.029321,0.002315,0.074846,0.028164,-0.077160,0.036095
0.25,0.005401,0.006173,0.006944,0.198302,0.007716,0.097994,0.002315,0.005401,0.005401,0.176698,0.002315,0.198302,0.059414,0.031250,0.077506
0.1,0.038580,0.003858,0.007716,0.007716,0.003086,0.027778,0.199074,0.146605,0.059414,0.021605,0.003086,0.199074,0.059799,0.000386,0.072074
0.05,0.250772,0.136574,0.171296,0.347222,0.000772,0.017747,0.004630,0.003086,0.023148,0.006173,0.000772,0.347222,0.109118,0.049318,0.128113
0.01,0.122685,0.023920,0.000000,0.088735,0.172840,0.004630,0.030093,0.031636,0.000772,0.111111,0.000000,0.172840,0.063272,-0.045846,0.068239
0.005,0.023148,0.013117,0.012346,0.102623,0.020833,0.294753,0.233796,0.023148,0.005401,0.037037,0.005401,0.294753,0.088863,0.025592,0.107835
0.001,0.018519,0.297068,0.311728,0.167438,0.006173,0.004630,0.264660,0.122685,0.087963,0.135802,0.004630,0.311728,0.144419,0.055556,0.117469



Wall time: 1min 5s


# Optical Digit Recognition Dataset

In [20]:
%%time

# Define the headings to be used
headings = list(range(0, 64))
headings.append('label')

# Read in the official training and testing data separately
opt_training_data = pd.read_csv('datasets/Optical_Digit_Recognition/optdigits.tra', names=headings)
opt_testing_data = pd.read_csv('datasets/Optical_Digit_Recognition/optdigits.tes', names=headings)

opt_data = opt_training_data.append(opt_testing_data)

# Apply some pre-processing
for heading in headings:
    opt_data[heading] = opt_data[heading].astype('int').apply(pd.to_numeric, downcast="unsigned")
    
# Find out which features contain missing (NaN) values
nan_features = opt_data.columns[opt_data.isnull().any()]
nan_features_count = opt_data[nan_features].isnull().sum()

print("Number of records with NaN values, per feature:")
display(nan_features_count)
print()

# Get the number of records in the data set
n_records_before = opt_data.shape[0]

# Drop the rows with missing values, since they occur in multiple features
opt_data = opt_data.dropna()

# Get the number of records in the data set
n_records_after = opt_data.shape[0]

# Print how many records containing NaN values got dropped
n_records_dropped = n_records_before - n_records_after
print(n_records_dropped, "records were dropped due to missing values.")
print("This is", round(n_records_dropped / n_records_before * 100, 1), "% of the entire data set.")
print("The resulting data set contains", n_records_after, "records.")

# Reset the indices
opt_data = opt_data.reset_index(drop=True)

# Print a newline to separate outputs
print()

Number of records with NaN values, per feature:


Series([], dtype: float64)


0 records were dropped due to missing values.
This is 0.0 % of the entire data set.
The resulting data set contains 5620 records.

Wall time: 1.8 s


In [21]:
display(opt_data.head())

opt_data.shape

,0,1,2,3,4,5,6,7,8,9,...,55,56,57,58,59,60,61,62,63,label
0,0,1,6,15,12,1,0,0,0,7,...,0,0,0,6,14,7,1,0,0,0
1,0,0,10,16,6,0,0,0,0,7,...,0,0,0,10,16,15,3,0,0,0
2,0,0,8,15,16,13,0,0,0,1,...,0,0,0,9,14,0,0,0,0,7
3,0,0,0,3,11,16,0,0,0,0,...,0,0,0,0,1,15,2,0,0,4
4,0,0,5,14,4,0,0,0,0,0,...,0,0,0,4,12,14,7,0,0,6


(5620, 65)

In [22]:
%%time

run_classifier(opt_data, 'label', 'datasets/Optical_Digit_Recognition/buckets_optdigits.csv')

Timings (in seconds) for original algorithm with e = None :


,fold 1,fold 2,fold 3,fold 4,fold 5,fold 6,fold 7,fold 8,fold 9,fold 10,mean,std
train,0.0419719,0.038976,0.037977,0.0369761,0.0399742,0.038976,0.0389752,0.0429754,0.0389748,0.038976,0.039475,0.001687
test,0.0969455,0.0959411,0.0989711,0.0977829,0.0999396,0.095942,0.0949419,0.0969403,0.0989397,0.0944541,0.097080,0.001726
total,0.138917,0.134917,0.136948,0.134759,0.139914,0.134918,0.133917,0.139916,0.137914,0.13343,0.136555,0.002355



Timings (in seconds) for enhanced algorithm with e = None :


,fold 1,fold 2,fold 3,fold 4,fold 5,fold 6,fold 7,fold 8,fold 9,fold 10,mean,std
train,0.0369775,0.0389559,0.0379758,0.0369761,0.0379741,0.0399754,0.0399747,0.0459731,0.0399737,0.0399747,0.039473,0.002459
test,0.100937,0.0944471,0.0969405,0.0929444,0.105937,0.103935,0.114931,0.118926,0.119929,0.0959401,0.104487,0.009670
total,0.137914,0.133403,0.134916,0.12992,0.143911,0.143911,0.154905,0.164899,0.159902,0.135915,0.143960,0.011419



Timings (in seconds) for original algorithm with e = 1.0 :


...


Original Algorithm:


,accuracy 1,accuracy 2,accuracy 3,accuracy 4,accuracy 5,accuracy 6,accuracy 7,accuracy 8,accuracy 9,accuracy 10,min,max,mean,mean_diff,std
None,0.727758,0.718861,0.676157,0.747331,0.729537,0.715302,0.706406,0.724199,0.715302,0.692171,0.676157,0.747331,0.714709,NaN,0.022686
1.0,0.741993,0.733096,0.658363,0.733096,0.725979,0.715302,0.717082,0.685053,0.702847,0.683274,0.658363,0.741993,0.708037,-0.006673,0.193041
0.75,0.711744,0.747331,0.656584,0.759786,0.740214,0.695730,0.676157,0.715302,0.720641,0.640569,0.640569,0.759786,0.705368,-0.002669,0.193451
0.5,0.637011,0.718861,0.715302,0.708185,0.745552,0.738434,0.727758,0.669039,0.642349,0.669039,0.637011,0.745552,0.696174,-0.009193,0.192566
0.25,0.661922,0.567616,0.558719,0.653025,0.533808,0.606762,0.608541,0.580071,0.629893,0.555160,0.533808,0.661922,0.595937,-0.100237,0.191202
0.1,0.482206,0.539146,0.467972,0.533808,0.450178,0.572954,0.524911,0.352313,0.619217,0.250890,0.250890,0.619217,0.471975,-0.123962,0.197512
0.05,0.270463,0.450178,0.355872,0.359431,0.450178,0.459075,0.354093,0.320285,0.274021,0.425267,0.270463,0.459075,0.370700,-0.101275,0.144298
0.01,0.147687,0.199288,0.302491,0.272242,0.234875,0.236655,0.215302,0.314947,0.104982,0.113879,0.104982,0.314947,0.213523,-0.157177,0.123588
0.005,0.259786,0.188612,0.149466,0.145907,0.206406,0.233096,0.213523,0.291815,0.256228,0.243772,0.145907,0.291815,0.218861,0.005338,0.075158
0.001,0.113879,0.110320,0.172598,0.213523,0.215302,0.213523,0.161922,0.241993,0.195730,0.183274,0.110320,0.241993,0.181198,-0.037663,0.073570



Enhanced Algorithm:


,accuracy 1,accuracy 2,accuracy 3,accuracy 4,accuracy 5,accuracy 6,accuracy 7,accuracy 8,accuracy 9,accuracy 10,min,max,mean,mean_diff,std
None,0.727758,0.718861,0.676157,0.747331,0.729537,0.715302,0.706406,0.724199,0.715302,0.692171,0.676157,0.747331,0.714709,NaN,0.022686
1.0,0.053381,0.071174,0.113879,0.131673,0.201068,0.128114,0.120996,0.249110,0.128114,0.092527,0.053381,0.249110,0.132711,-0.581999,0.200886
0.75,0.154804,0.137011,0.092527,0.097865,0.099644,0.144128,0.056940,0.140569,0.078292,0.078292,0.056940,0.154804,0.107651,-0.025059,0.049094
0.5,0.087189,0.106762,0.186833,0.119217,0.154804,0.042705,0.128114,0.131673,0.174377,0.080071,0.042705,0.186833,0.120107,0.012456,0.054674
0.25,0.092527,0.048043,0.108541,0.081851,0.069395,0.135231,0.199288,0.096085,0.094306,0.144128,0.048043,0.199288,0.109727,-0.010380,0.056844
0.1,0.056940,0.156584,0.177936,0.138790,0.106762,0.129893,0.104982,0.071174,0.126335,0.101423,0.056940,0.177936,0.117141,0.007414,0.048602
0.05,0.195730,0.172598,0.092527,0.112100,0.096085,0.177936,0.035587,0.108541,0.067616,0.131673,0.035587,0.195730,0.118476,0.001335,0.061368
0.01,0.104982,0.122776,0.160142,0.161922,0.065836,0.165480,0.096085,0.117438,0.140569,0.106762,0.065836,0.165480,0.122776,0.004300,0.046181
0.005,0.014235,0.137011,0.129893,0.024911,0.131673,0.090747,0.040925,0.160142,0.024911,0.085409,0.014235,0.160142,0.084520,-0.038256,0.062879
0.001,0.144128,0.108541,0.179715,0.115658,0.101423,0.117438,0.044484,0.065836,0.153025,0.104982,0.044484,0.179715,0.113286,0.028766,0.048009



Wall time: 46 s


# Skin Dataset

In [23]:
%%time

# Define the headings to be used
headings = ['B', 'G', 'R', 'label']

# Read in the official training and testing data separately
skin_data = pd.read_csv('datasets/Skin/Skin_NonSkin.txt', names=headings, delimiter='\t')

# Apply some pre-processing
for heading in headings:
    skin_data[heading] = skin_data[heading].astype('int').apply(pd.to_numeric, downcast="unsigned")
    
# Find out which features contain missing (NaN) values
nan_features = skin_data.columns[skin_data.isnull().any()]
nan_features_count = skin_data[nan_features].isnull().sum()

print("Number of records with NaN values, per feature:")
display(nan_features_count)
print()

# Get the number of records in the data set
n_records_before = skin_data.shape[0]

# Drop the rows with missing values, since they occur in multiple features
skin_data = skin_data.dropna()

# Get the number of records in the data set
n_records_after = skin_data.shape[0]

# Print how many records containing NaN values got dropped
n_records_dropped = n_records_before - n_records_after
print(n_records_dropped, "records were dropped due to missing values.")
print("This is", round(n_records_dropped / n_records_before * 100, 1), "% of the entire data set.")
print("The resulting data set contains", n_records_after, "records.")

# Reset the indices
skin_data = skin_data.reset_index(drop=True)

# Print a newline to separate outputs
print()

Number of records with NaN values, per feature:


Series([], dtype: float64)


0 records were dropped due to missing values.
This is 0.0 % of the entire data set.
The resulting data set contains 245057 records.

Wall time: 4.72 s


In [24]:
display(skin_data.head())

skin_data.shape

,B,G,R,label
0,74,85,123,1
1,73,84,122,1
2,72,83,121,1
3,70,81,119,1
4,70,81,119,1


(245057, 4)

In [25]:
%%time

run_classifier(skin_data, 'label', 'datasets/Skin/buckets_skin.csv')

Timings (in seconds) for original algorithm with e = None :


,fold 1,fold 2,fold 3,fold 4,fold 5,fold 6,fold 7,fold 8,fold 9,fold 10,mean,std
train,0.0939553,0.11791,0.141306,0.102936,0.0989382,0.101938,0.096453,0.0959413,0.124923,0.0959389,0.107024,0.015001
test,0.06496,0.0739541,0.0739532,0.096941,0.057965,0.0569658,0.0609627,0.0583649,0.0699577,0.0599632,0.067399,0.011607
total,0.158915,0.191864,0.21526,0.199877,0.156903,0.158904,0.157416,0.154306,0.194881,0.155902,0.174423,0.022053



Timings (in seconds) for enhanced algorithm with e = None :


,fold 1,fold 2,fold 3,fold 4,fold 5,fold 6,fold 7,fold 8,fold 9,fold 10,mean,std
train,0.0939415,0.0952511,0.0939448,0.0938175,0.093941,0.0937545,0.0949464,0.0952358,0.0969398,0.0939658,0.094574,0.000972
test,0.0565419,0.0589914,0.061964,0.059963,0.0569658,0.0609682,0.0579643,0.0649834,0.0604777,0.0582898,0.059711,0.002414
total,0.150483,0.154243,0.155909,0.15378,0.150907,0.154723,0.152911,0.160219,0.157418,0.152256,0.154285,0.002828



Timings (in seconds) for original algorithm with e = 1.0 :


...


Original Algorithm:


,accuracy 1,accuracy 2,accuracy 3,accuracy 4,accuracy 5,accuracy 6,accuracy 7,accuracy 8,accuracy 9,accuracy 10,min,max,mean,mean_diff,std
None,0.923730,0.923730,0.924424,0.923444,0.922628,0.927443,0.921037,0.925893,0.923689,0.923099,0.921037,0.927443,0.923966,NaN,0.002006
1.0,0.925852,0.923526,0.924668,0.923322,0.923444,0.927076,0.921118,0.926872,0.923689,0.923140,0.921118,0.927076,0.924242,0.000275,0.246948
0.75,0.923811,0.924138,0.924668,0.923444,0.923322,0.927362,0.921649,0.928300,0.924097,0.923099,0.921649,0.928300,0.924487,0.000245,0.247023
0.5,0.924179,0.923403,0.924791,0.924220,0.923322,0.927403,0.922465,0.927851,0.924097,0.923466,0.922465,0.927851,0.924626,0.000139,0.247086
0.25,0.927607,0.923567,0.924628,0.926709,0.923199,0.930790,0.921853,0.926505,0.923199,0.922691,0.921853,0.930790,0.925282,0.000656,0.247134
0.1,0.925240,0.924260,0.918180,0.914181,0.927933,0.931687,0.918058,0.924016,0.918221,0.921549,0.914181,0.931687,0.922433,-0.002850,0.247355
0.05,0.929484,0.929647,0.926505,0.918670,0.928423,0.899735,0.924628,0.909284,0.912752,0.929259,0.899735,0.929647,0.919814,-0.002619,0.246762
0.01,0.547398,0.825831,0.326913,0.898184,0.780290,0.544175,0.791308,0.788982,0.789063,0.839385,0.326913,0.898184,0.696386,-0.223428,0.310815
0.005,0.409141,0.792573,0.286023,0.792206,0.790410,0.548255,0.452153,0.829096,0.550786,0.253101,0.253101,0.829096,0.565495,-0.130891,0.284730
0.001,0.795266,0.206325,0.792287,0.452561,0.209590,0.206815,0.208692,0.211018,0.789594,0.798221,0.206325,0.798221,0.472910,-0.092585,0.308332



Enhanced Algorithm:


,accuracy 1,accuracy 2,accuracy 3,accuracy 4,accuracy 5,accuracy 6,accuracy 7,accuracy 8,accuracy 9,accuracy 10,min,max,mean,mean_diff,std
None,0.923730,0.923730,0.924424,0.923444,0.922628,0.927443,0.921037,0.925893,0.923689,0.923099,0.921037,0.927443,0.923966,NaN,0.002006
1.0,0.795266,0.793675,0.232238,0.216078,0.236850,0.206815,0.234564,0.211018,0.789063,0.798221,0.206815,0.798221,0.459902,-0.464064,0.367500
0.75,0.204734,0.793675,0.223057,0.207794,0.209590,0.787064,0.208692,0.211018,0.789063,0.201779,0.201779,0.793675,0.402660,-0.057242,0.291019
0.5,0.781432,0.793675,0.207713,0.792206,0.784615,0.206815,0.791308,0.788982,0.210937,0.207735,0.206815,0.793675,0.547159,0.144499,0.295693
0.25,0.204734,0.206325,0.223057,0.207794,0.236850,0.793185,0.208692,0.788982,0.789063,0.798221,0.204734,0.798221,0.454988,-0.092171,0.311495
0.1,0.782738,0.793675,0.207713,0.792206,0.209590,0.206815,0.791308,0.788982,0.780290,0.207735,0.206815,0.793675,0.546795,0.091807,0.300999
0.05,0.204734,0.206325,0.792287,0.792206,0.209590,0.219996,0.779922,0.211018,0.210937,0.798221,0.204734,0.798221,0.452349,-0.094446,0.312292
0.01,0.204734,0.793675,0.217874,0.207794,0.222567,0.793185,0.791308,0.788982,0.233626,0.798221,0.204734,0.798221,0.504577,0.052227,0.303247
0.005,0.795266,0.784738,0.223057,0.207794,0.209590,0.206815,0.217996,0.211018,0.789063,0.201779,0.201779,0.795266,0.403680,-0.100897,0.295786
0.001,0.795266,0.771271,0.792287,0.792206,0.209590,0.206815,0.208692,0.775882,0.789063,0.792265,0.206815,0.795266,0.594618,0.190938,0.284044



Wall time: 5min 44s


# SPECT Heart Dataset

In [26]:
%%time

# Define the headings to be used
headings = ['overall_diagnosis', 'F1', 'F2', 'F3', 'F4', 'F5', 'F6', 'F7', 'F8', 'F9', 'F10', 'F11', 'F12', 'F13', 'F14', 'F15', 'F16', 'F17', 'F18', 'F19', 'F20', 'F21', 'F22']

# Read in the official training and testing data separately
spect_training_data = pd.read_csv('datasets/SPECT_Heart/SPECT.train', names=headings)
spect_testing_data = pd.read_csv('datasets/SPECT_Heart/SPECT.test', names=headings)

spect_data = spect_training_data.append(spect_testing_data)

# Apply some pre-processing
for heading in headings:
    spect_data[heading] = spect_data[heading].astype('int').apply(pd.to_numeric, downcast="unsigned")
    
# Find out which features contain missing (NaN) values
nan_features = spect_data.columns[spect_data.isnull().any()]
nan_features_count = spect_data[nan_features].isnull().sum()

print("Number of records with NaN values, per feature:")
display(nan_features_count)
print()

# Get the number of records in the data set
n_records_before = spect_data.shape[0]

# Drop the rows with missing values, since they occur in multiple features
spect_data = spect_data.dropna()

# Get the number of records in the data set
n_records_after = spect_data.shape[0]

# Print how many records containing NaN values got dropped
n_records_dropped = n_records_before - n_records_after
print(n_records_dropped, "records were dropped due to missing values.")
print("This is", round(n_records_dropped / n_records_before * 100, 1), "% of the entire data set.")
print("The resulting data set contains", n_records_after, "records.")

# Reset the indices
spect_data = spect_data.reset_index(drop=True)

# Print a newline to separate outputs
print()

Number of records with NaN values, per feature:


Series([], dtype: float64)


0 records were dropped due to missing values.
This is 0.0 % of the entire data set.
The resulting data set contains 267 records.

Wall time: 66 ms


In [27]:
display(spect_data.head())

spect_data.shape

,overall_diagnosis,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F13,F14,F15,F16,F17,F18,F19,F20,F21,F22
0,1,0,0,0,1,0,0,0,1,1,...,1,1,0,0,0,0,0,0,0,0
1,1,0,0,1,1,0,0,0,1,1,...,1,1,0,0,0,0,0,0,0,1
2,1,1,0,1,0,1,0,0,1,0,...,1,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,1
4,1,0,0,0,0,0,0,0,1,0,...,1,0,1,1,0,0,0,0,0,0


(267, 23)

In [28]:
%%time

run_classifier(spect_data, 'overall_diagnosis', 'datasets/SPECT_Heart/buckets_SPEC.csv')

Timings (in seconds) for original algorithm with e = None :


,fold 1,fold 2,fold 3,fold 4,fold 5,fold 6,fold 7,fold 8,fold 9,fold 10,mean,std
train,0.0379758,0.0129921,0.0129902,0.0139914,0.0129919,0.0129938,0.0129919,0.0129921,0.0119927,0.0129919,0.015490,0.007508
test,0.0179894,0.0129936,0.0119927,0.0119927,0.0119927,0.0124972,0.0129921,0.0119925,0.0129919,0.011992,0.012943,0.001738
total,0.0559652,0.0259857,0.0249829,0.025984,0.0249846,0.025491,0.025984,0.0249846,0.0249846,0.0249839,0.028433,0.009188



Timings (in seconds) for enhanced algorithm with e = None :


,fold 1,fold 2,fold 3,fold 4,fold 5,fold 6,fold 7,fold 8,fold 9,fold 10,mean,std
train,0.0119925,0.0129921,0.0119927,0.0129943,0.0129924,0.0129919,0.0124993,0.0119927,0.0129919,0.0129921,0.012643,0.000450
test,0.0129921,0.0119927,0.0129919,0.0139894,0.0119932,0.0119927,0.0119927,0.0129919,0.0129921,0.0119925,0.012592,0.000662
total,0.0249846,0.0249848,0.0249846,0.0269837,0.0249856,0.0249846,0.024492,0.0249846,0.025984,0.0249846,0.025235,0.000679



Timings (in seconds) for original algorithm with e = 1.0 :


...


Original Algorithm:


,accuracy 1,accuracy 2,accuracy 3,accuracy 4,accuracy 5,accuracy 6,accuracy 7,accuracy 8,accuracy 9,accuracy 10,min,max,mean,mean_diff,std
None,0.576923,0.653846,0.692308,0.500000,0.461538,0.615385,0.615385,0.615385,0.538462,0.636364,0.461538,0.692308,0.588287,NaN,0.077990
1.0,0.538462,0.692308,0.692308,0.653846,0.461538,0.692308,0.653846,0.807692,0.769231,0.575758,0.461538,0.807692,0.650544,0.062257,0.192093
0.75,0.653846,0.730769,0.769231,0.730769,0.538462,0.653846,0.538462,0.807692,0.653846,0.484848,0.484848,0.807692,0.654526,0.003982,0.205540
0.5,0.653846,0.730769,0.653846,0.538462,0.423077,0.653846,0.692308,0.807692,0.346154,0.787879,0.346154,0.807692,0.620144,-0.034382,0.234158
0.25,0.576923,0.500000,0.538462,0.730769,0.615385,0.423077,0.576923,0.769231,0.461538,0.666667,0.423077,0.769231,0.587607,-0.032537,0.202095
0.1,0.423077,0.615385,0.615385,0.230769,0.692308,0.807692,0.576923,0.692308,0.307692,0.606061,0.230769,0.807692,0.550505,-0.037102,0.245731
0.05,0.538462,0.538462,0.576923,0.576923,0.423077,0.461538,0.615385,0.230769,0.269231,0.363636,0.230769,0.615385,0.453380,-0.097125,0.200185
0.01,0.269231,0.307692,0.384615,0.230769,0.384615,0.384615,0.500000,0.346154,0.230769,0.636364,0.230769,0.636364,0.378497,-0.074883,0.180215
0.005,0.346154,0.346154,0.423077,0.230769,0.346154,0.461538,0.384615,0.346154,0.230769,0.636364,0.230769,0.636364,0.384907,0.006410,0.162694
0.001,0.269231,0.500000,0.384615,0.269231,0.307692,0.423077,0.461538,0.230769,0.269231,0.363636,0.230769,0.500000,0.350816,-0.034091,0.139096



Enhanced Algorithm:


,accuracy 1,accuracy 2,accuracy 3,accuracy 4,accuracy 5,accuracy 6,accuracy 7,accuracy 8,accuracy 9,accuracy 10,min,max,mean,mean_diff,std
None,0.576923,0.653846,0.692308,0.500000,0.461538,0.615385,0.615385,0.615385,0.538462,0.636364,0.461538,0.692308,0.588287,NaN,0.077990
1.0,0.461538,0.423077,0.423077,0.230769,0.230769,0.461538,0.576923,0.346154,0.461538,0.636364,0.230769,0.636364,0.426573,-0.161713,0.207024
0.75,0.576923,0.500000,0.423077,0.615385,0.423077,0.423077,0.423077,0.346154,0.500000,0.454545,0.346154,0.615385,0.470571,0.043998,0.142557
0.5,0.230769,0.384615,0.576923,0.230769,0.538462,0.423077,0.423077,0.576923,0.269231,0.363636,0.230769,0.576923,0.402098,-0.068473,0.180658
0.25,0.461538,0.500000,0.384615,0.230769,0.500000,0.384615,0.307692,0.346154,0.269231,0.363636,0.230769,0.500000,0.373252,-0.028846,0.142216
0.1,0.269231,0.307692,0.538462,0.653846,0.423077,0.307692,0.423077,0.384615,0.538462,0.393939,0.269231,0.653846,0.430264,0.057012,0.161287
0.05,0.230769,0.461538,0.500000,0.346154,0.384615,0.269231,0.384615,0.192308,0.653846,0.393939,0.192308,0.653846,0.388598,-0.041667,0.185727
0.01,0.500000,0.615385,0.461538,0.269231,0.307692,0.423077,0.384615,0.692308,0.730769,0.363636,0.269231,0.730769,0.479021,0.090423,0.191058
0.005,0.269231,0.692308,0.461538,0.192308,0.269231,0.576923,0.384615,0.769231,0.307692,0.575758,0.192308,0.769231,0.455031,-0.023990,0.236976
0.001,0.461538,0.384615,0.500000,0.230769,0.384615,0.461538,0.500000,0.384615,0.653846,0.393939,0.230769,0.653846,0.436674,-0.018357,0.173338



Wall time: 11.1 s
